## Description
There are three main parts:
1. Preliminary processing for munging
2. Munging for LDSC
3. LDSC to calculate heritability

## 1. Preliminary
This preliminary section will be ran in `python3`.

Let's read `MDD_clean.csv` and convert to a `.txt` file ready to be munged.

In [1]:
import pandas as pd

# Step 1: Read the CSV file
df = pd.read_csv("MDD_clean.csv")

# Step 2: Select and reorder desired columns
df_subset = pd.DataFrame({
    "SNP": df["SNP"],
    "A1": df["A1"],
    "A2": df["A2"],
    "P": df["P"],
    "Z": df["Z"],
    "MAF": df["MAF"]
})

# Step 3: Save as tab-delimited text file
df_subset.to_csv("MDD_clean_munge.txt", sep="\t", index=False)


In [2]:
df_subset.head()

,A1,A2,MAF,P,SNP,Z
0,A,G,0.843911,0.9284,rs12238997,-0.093505
1,C,G,0.193262,0.8311,rs4951859,-0.216722
2,T,C,0.944587,0.3532,rs148120343,-0.929571
3,T,C,0.836911,0.9760,rs142557973,0.029407
4,T,C,0.838911,0.9207,rs141242758,-0.098088


## 2. Munging for LDSC
The rest of the notebook will be ran in `python2`.

In [3]:
%%bash

# Input parameters
munge='/gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/munge_sumstats.py' # Path to munge script
gwas='MDD_clean_munge.txt' # Path to GWAS summary statistics
path_out='./' # output path
file_out='munge_MDD' # prefix of munged output
## required columns
snp='SNP' # rsID
a1='A1' # effect allele
a2='A2' # non-effect allele
frq='MAF' # frequency of effect allele
p='P' # p-value
effect_size='Z' # effect sizes, can be either odds ratio or z statistic
null_value=$([ "$effect_size" = "OR" ] && echo 1 || echo 0) # null value of the effect size
## for optional column, if you don't have it, manually add sample size
N=674452
## Set threshold for quality control on minor allele frequency
maf=0.01 # default

# Run
python ${munge} \
    --sumstats ${gwas} \
    --snp ${snp} \
    --a1 ${a1} \
    --a2 ${a2} \
    --frq ${frq} \
    --p ${p} \
    --N ${N} \
    --signed-sumstats ${effect_size},${null_value} \
    --maf-min ${maf} \
    --out ${path_out}/${file_out}

*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./munge_sumstats.py \
--signed-sumstats Z,0 \
--out .//munge_MDD \
--frq MAF \
--N 674452.0 \
--a1 A1 \
--a2 A2 \
--snp SNP \
--sumstats MDD_clean_munge.txt \
--p P 

Interpreting column names as follows:
A1:	Allele 1, interpreted as ref allele for signed sumstat.
P:	p-Value
A2:	Allele 2, interpreted as non-ref allele for signed sumstat.
SNP:	Variant ID (e.g., rs number)
MAF:	Allele frequency
Z:	Directional summary statistic as specified by --signed-sumstats.

Reading sumstats from MDD_clean_munge.txt into memory 5000000 SNPs at a time.
.. done
Read 7266505 SNPs from --sumstats file.
Removed 0 SNPs with missing values.
Removed 0 SNPs with I

### 2.1 Briefly view outputted munged file

In [2]:
%%bash
zcat munge_MDD.sumstats.gz|head

SNP	A1	A2	Z	N
rs12238997	A	G	-0.090	674452.000
rs148120343	T	C	-0.928	674452.000
rs142557973	T	C	0.030	674452.000
rs141242758	T	C	-0.100	674452.000
rs3094315	G	A	-0.379	674452.000
rs3131972	A	G	-0.375	674452.000
rs3115860	C	A	0.635	674452.000
rs2073813	G	A	0.376	674452.000
rs3131969	A	G	-0.417	674452.000



gzip: stdout: Broken pipe


## 3. LDSC to calculate heritability

In [4]:
%%bash

# Input parameters
ldsc='/gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/ldsc.py' # path to ldsc.py
ldscore='/gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/LDscore/' # folder for ldscores and correponding regression weights
gwas='munge_MDD.sumstats.gz' # path to munged gwas summary statistics
output_path='./' # directory of output files

python ${ldsc} \
    --h2 ${gwas} \
    --ref-ld-chr ${ldscore}LDscore.@ \
    --w-ld-chr ${ldscore}LDscore.@ \
    --out ${output_path}ldsc_MDD

*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--h2 munge_MDD.sumstats.gz \
--ref-ld-chr /gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/LDscore/LDscore.@ \
--out ./ldsc_MDD \
--w-ld-chr /gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/LDscore/LDscore.@ 

Beginning analysis at Thu Jul 17 09:28:00 2025
Reading summary statistics from munge_MDD.sumstats.gz ...
Read summary statistics for 6168307 SNPs.
Reading reference panel LD Score from /gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/LDscore/LDscore.[1-22] ... (ldscore_fromlist)
Read reference panel LD Scores for 1190321 SNPs.
Removing partitioned LD Scores with zero variance.
Reading regression weight LD Score from /gpfs/gibbs/pro

/gpfs/gibbs/project/bdsi/shared/Genetics/ldsc/ldscore/irwls.py:161: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef = np.linalg.lstsq(x, y)
